# Solving search problem.

An introduction to different methods for findings paths, including:

	adjacency matrix
	BFS
    find loops
	DFS
      DFScan
	bidi-BFS
	TSP
	[critical path method]
	


First things first. Let's load the imports for this chapter

In [6]:
from graph import Graph

## The adjacency matrix

an adjacency matrix is a square matrix used to represent a finite graph. The elements of the matrix indicate whether pairs of vertices are adjacent or not in the graph.

   The distance from a node to itself is 0 and distance from a node to
    an unconnected node is defined to be infinite. This does not mean that there
    is no path from a node to another via other nodes.

```
Example:
    g = Graph(from_dict=
        {1: {2: 3, 3: 8, 5: -4},
         2: {4: 1, 5: 7},
         3: {2: 4},
         4: {1: 2, 3: -5},
         5: {4: 6}})

    adjacency_matrix(g)
    {1: {1: 0, 2: 3, 3: 8, 4: inf, 5: -4},
     2: {1: inf, 2: 0, 3: inf, 4: 1, 5: 7},
     3: {1: inf, 2: 4, 3: 0, 4: inf, 5: inf},
     4: {1: 2, 2: inf, 3: -5, 4: 0, 5: inf},
     5: {1: inf, 2: inf, 3: inf, 4: 6, 5: 0}}
```

The adjacency matrix is very helpful when we want to compute the all pairs shortest path.

Find the cost of the shortest path between every pair of vertices in a
    weighted graph. Uses the Floyd-Warshall algorithm.

    Example:
        inf = float('inf')
        g = Graph(from_dict=(
            {0: {0: 0,   1: 1,   2: 4},
             1: {0: inf, 1: 0,   2: 2},
             2: {0: inf, 1: inf, 2: 0}})

        fw(g)
        {0: {0: 0,   1: 1,   2: 3},
        1: {0: inf, 1: 0,   2: 2},
        2: {0: inf, 1: inf, 2: 0}}

        h = {1: {2: 3, 3: 8, 5: -4},
             2: {4: 1, 5: 7},
             3: {2: 4},
             4: {1: 2, 3: -5},
             5: {4: 6}}

        fw(adj(h)) #
            {1: {1: 0, 2:  1, 3: -3, 4: 2, 5: -4},
             2: {1: 3, 2:  0, 3: -4, 4: 1, 5: -1},
             3: {1: 7, 2:  4, 3:  0, 4: 5, 5:  3},
             4: {1: 2, 2: -1, 3: -5, 4: 0, 5: -2},
             5: {1: 8, 2:  5, 3:  1, 4: 6, 5:  0}}


### Distance maps

As you can see the adjacency matrix computes the whole graph, but often this isn't necessary. So I invented the distance map as "light weight" version of the adjacency matrix. Here is how it works:

We have a 4x4 graph like this:

    1 -> 2 -> 3 -> 4
    |    |    |    |
    v    v    v    v
    5 -> 6 -> 7 -> 8
    |    |    |    |
    v    v    v    v
    9 -> 10-> 11-> 12
    |    |    |    |
    v    v    v    v
    13-> 14-> 15-> 16

And we would like to go the shortest distance between _some_ points.

Arbitrarily we pick 3 or 5 as starts and decide to go to either 12 or 14.

The objective is to choose ANY shortest path.

In [16]:
edges = [
      (1, 2, 1),   (2, 3, 1),   (3, 4, 1),   (1, 5, 1),
      (5, 6, 1),   (6, 7, 1),   (7, 8, 1),   (2, 6, 1),
      (3, 7, 1),   (4, 8, 1),   (5, 9, 1),  (9, 10, 1),
    (10, 11, 1), (11, 12, 1),  (6, 10, 1),  (7, 11, 1),
     (8, 12, 1),  (9, 13, 1), (13, 14, 1), (14, 15, 1),
    (15, 16, 1), (10, 14, 1), (11, 15, 1), (12, 16, 1)
]
g = Graph()
for s, e, d in edges:
    g.add_edge(s, e, d, bidirectional=True)
    
g.distance_map(starts=[3, 5], ends=[12, 14])


{3: 0,
 5: 0,
 2: 1,
 4: 1,
 7: 1,
 1: 1,
 6: 1,
 9: 1,
 8: 2,
 11: 2,
 10: 2,
 13: 2,
 12: 3,
 15: 3,
 14: 3,
 16: 4}

The distance map returns a dictionary with the distance from 3 and 5 traveling towards 12 and 14. 

Think of this map as a landscape, where the `starts` are at the bottom and then you want to "walk" towards either 12 or 14 following the path of least resistance.

In my experience a lot of people are initially confused about the all-pairs shortest path, and it's usage.

Let's compare it with two other methods:

minsum: finds the mode(s) that have the smallest sum of distance to all other nodes

minmax: finds the node(s) with shortest distance to all other nodes. 


## Breadth First Search (BFS)

Breadth-first search (BFS) is an algorithm for searching a tree data structure for a node that satisfies a given property. It starts at the tree root and explores all nodes at the present depth prior to moving on to the nodes at the next depth level. Extra memory, usually a queue, is needed to keep track of the child nodes that were encountered but not yet explored.

A simple example is where you are searching for the nearest gas station on a map: You start with your current position and follow the roads in all directions until you meet an intersection. You repeat this "extension" until you find a gas station.

BFS on a map is very easy to understand. But what if you don't have a map? What if you only know where you can go?

For example, in a chess endgame a chess engine may build the game tree from the current position by applying all possible moves, and use breadth-first search to find a win position for white. Implicit trees (such as game trees or other problem-solving trees) may be of infinite size; breadth-first search is guaranteed to find a solution node if one exists.

A slightly simpler case is the search for a solution of the tile-slide puzzle.

We can describe the state of a system by it's options as a graph:

```
 A       B
[1] --- [2]
 |       |
 |       |
[4] --- [3]
```
With tiles on position `[1]` and `[2]` where A is in 1 and B is in 2.
Let's find a way for A to move to 3 and B to move to 4.

First we create the graph:

In [8]:
g = Graph()
for s,e in zip([1,2,3,4], [2,3,4,1]):
    g.add_edge(s,e,1,bidirectional=True)

Then we define the `initial state` and `end state`:

In [9]:
s1 = (('A',1), ('B',2))

In [10]:
end = (('A',3),('B',4))

Now `A` can't move from 1 via 2 to 3, as `B` blocks the way.
Similarly `B` can't move from 2 via 1 to 3, as `A` blocks the way.
So we need generate a tree using BFS as foundation for our search.

the search will look like this:

![search tree](images/search_tree.png)

but as you see there is some redunancy in the search tree, it is better to create a graph of movements:

![movements](images/movement_graph.png)

We can now search for the shortest path from the `initial state` to the `end state` in the movements graph:

This is the code from graph-theory

In [11]:
from graph import breadth_first_search
import inspect

In [13]:
print(inspect.getsource(breadth_first_search))

def breadth_first_search(graph, start, end):
    """ Determines the path from start to end with fewest nodes.
    :param graph: class Graph
    :param start: start node
    :param end: end node
    :return: path
    """
    if not isinstance(graph, (BasicGraph, Graph, Graph3D)):
        raise TypeError(f"Expected BasicGraph, Graph or Graph3D, not {type(graph)}")
    if start not in graph:
        raise ValueError(f"{start} not in graph")
    if end not in graph:
        raise ValueError(f"{end} not in graph")

    visited = {start: None}
    q = deque([start])
    while q:
        node = q.popleft()
        if node == end:
            path = deque()
            while node is not None:
                path.appendleft(node)
                node = visited[node]
            return list(path)
        for next_node in graph.nodes(from_node=node):
            if next_node not in visited:
                visited[next_node] = node
                q.append(next_node)
    return []



**Explanation of the code**

`breadth_first_search` starts with the `start` (our `initial state`) and `end` (our `end state`) and puts `start` into a `deque`.

```
def breadth_first_search(graph, start, end):
    visited = {start: None}
    q = deque([start])
```
Then it enters the `while` loop and runs as long as there's node in the queue.
```
    while q:
        node = q.popleft()
        if node == end:
```
The algorithm then removes the first item from the queue using `popleft`, and checks whether it's the `end`. If it isn't, the search continues:

We take each node that the popped `node` is connected to and check if we have seen it before. If we have seen it before we just continue to the next node. If we haven't seen it before we add it to the queue.
```
        for next_node in graph.nodes(from_node=node):
            if next_node not in visited:
                visited[next_node] = node
                q.append(next_node)
```
Finally when the `end` is the node we are looking for we can terminate the search.

### finding loops

## Depth First Search (DFS)



## Bidirectional breadth first search (BidiBFS)

## Critical path method

The [critical path method](https://en.wikipedia.org/wiki/Critical_path_method) (CPM), or critical path analysis (CPA), is an algorithm for scheduling a set of project activities.

A critical path is determined by identifying the longest stretch of dependent activities and, commonly, measuring the time required to complete them from start to finish.

An example is shown below where the critical path constitutes the path ABCDE:

![w](images/cpm_wo_artificial_dependency.png)

We can load these values into a Graph as follows:

In [2]:
tasks = {'A': 10, 'B': 20, 'C': 5, 'D': 10, 'E': 20, 'F': 15, 'G': 5, 'H': 15}
dependencies = [
    ('A', 'B'),
    ('B', 'C'),
    ('C', 'D'),
    ('D', 'E'),
    ('A', 'F'),
    ('F', 'G'),
    ('G', 'E'),
    ('A', 'H'),
    ('H', 'E'),
]

g = Graph()
for n, d in tasks.items():
    g.add_node(n, obj=d)
for n1, n2 in dependencies:
    g.add_edge(n1, n2, 0)

And we can calculate the schedule and the length of the critical path as:

In [3]:
critical_path_length, schedule = g.critical_path()

In [4]:
print("The critical path has duration", critical_path_length)

The critical path has duration 65


In [5]:
print("The tasks are:")
from graph import Task
for task_id, task in sorted(schedule.items()):
    print(task_id, task)

The tasks are:
A Task('A', 10, 0, 0, 10, 10)
B Task('B', 20, 10, 10, 30, 30)
C Task('C', 5, 30, 30, 35, 35)
D Task('D', 10, 35, 35, 45, 45)
E Task('E', 20, 45, 45, 65, 65)
F Task('F', 15, 10, 25, 25, 40)
G Task('G', 5, 25, 40, 30, 45)
H Task('H', 15, 10, 30, 25, 45)


The properties of each `Task` are:

- task id
- duration 
- earliest start time
- latest start time
- earliest finish time
- latest finish time.

and the slack in the schedule can be calculated as:

In [6]:
slack = sum(t.slack for t in schedule.values())

print("The total slack in the schedule is", slack)

The total slack in the schedule is 50


### Minimising slack

In cases where the tasks are commodities, such as CPU time, it can be convenient to minimise the number of concurrently active resources.

As you may have noticed above in the diagram, the dependencies indicate that the graph has 3 paths at it's widest, whereby it would be logical to assign 3 CPUs to compute the tasks. However a little search can illustrate that it is possible to solve the tasks with 2 CPUs without extending the critical path.

This can be done by inserting artificial dependencies. Here is an example:

![wo](images/cpm_w_artificial_dependency.png)

The method to minimise the slack, is conveniently called `critical_path_minimize_for_slack` and this is how it is used:

In [7]:
g2 = g.critical_path_minimize_for_slack()

We can verify that the critical path length is the same, and we can verify that this schedule does indeed have less slack:

In [8]:
critical_path_length2, schedule2 = g2.critical_path()

slack2 = sum(t.slack for t in schedule2.values())

print("The total slack in the schedule was", slack, "and is now", slack2)

The total slack in the schedule was 50 and is now 0


In [9]:
print("The tasks remain the same, though with changed timings::")
from graph import Task
for task_id, task in sorted(schedule2.items()):
    print(task_id, task)

The tasks remain the same, though with changed timings::
A Task('A', 10, 0, 0, 10, 10)
B Task('B', 20, 10, 10, 30, 30)
C Task('C', 5, 30, 30, 35, 35)
D Task('D', 10, 35, 35, 45, 45)
E Task('E', 20, 45, 45, 65, 65)
F Task('F', 15, 25, 25, 40, 40)
G Task('G', 5, 40, 40, 45, 45)
H Task('H', 15, 10, 10, 25, 25)
